In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from tensorflow import keras
from tensorflow import lite
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [6]:
##Data Load
dataset = pd.read_csv('fake_or_real_news.csv')

##Take neccessarry columns
text = dataset['text']
label = dataset['label']

In [7]:
dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [8]:
dataset.shape


(6335, 4)

In [9]:
dataset.isnull().any()

Unnamed: 0    False
title         False
text          False
label         False
dtype: bool

In [10]:

x_train, x_test, y_train, y_test= train_test_split(text ,label , test_size=0.2)


In [11]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(x_train)
tfidf_test = tfidf_vectorizer.transform(x_test)

In [12]:
## model
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)
y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test, y_pred)
accuracy = score*100
print (accuracy)

94.79084451460142


In [13]:
pline = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                    ('nbmodel', MultinomialNB())])

In [14]:
#train data

pline.fit(x_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('nbmodel', MultinomialNB())])

In [15]:
score= pline.score(x_test, y_test)
accuracy = score*100
print ('accuracy',accuracy)

accuracy 81.61010260457773


In [16]:
##Report

pred =pline.predict(x_test)
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

        FAKE       0.98      0.66      0.79       660
        REAL       0.73      0.99      0.84       607

    accuracy                           0.82      1267
   macro avg       0.85      0.82      0.81      1267
weighted avg       0.86      0.82      0.81      1267



In [17]:
##COnfusion matrix
print(confusion_matrix(y_test, pred))

[[435 225]
 [  8 599]]


In [18]:
##Converting Model

keras_file='testing.h5'
keras.models.save_model(pline,keras_file)

AttributeError: 'Pipeline' object has no attribute '_is_graph_network'